In [ ]:
import os
import pandas as pd
import numpy as np
import re

In [ ]:
# Imported the dataset
from datasets import load_dataset
from datasets import Dataset

train_set1=load_dataset('ai4privacy/pii-masking-300k', split='train')
val_set=load_dataset('ai4privacy/pii-masking-300k', split='validation')

In [5]:
# Set up train_set2
filepath = os.path.join(os.getcwd(), 'maker_day_shrieyaa_stella_mini200_df.csv')
train_df2 = pd.read_csv(filepath)
train_set2 = Dataset.from_pandas(train_df2)

In [6]:
# Inspect the dataset
print(type(train_set2))
print(train_set2.shape)
val_set.shape

<class 'datasets.arrow_dataset.Dataset'>
(200, 3)


(47728, 9)

In [8]:
import random
# Select 100 random indices
# random_indices = random.sample(range(len(train_set1)), 10)
random_indices = random.sample(range(len(train_set2)), 10)

# Get the random samples
# random_samples = train_set1.select(random_indices)
random_samples = train_set2.select(random_indices)

print(random_samples)

Dataset({
    features: ['Unnamed: 0', 'unmasked_text', 'masked_text'],
    num_rows: 10
})


In [7]:
# Load model directly
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('lakshyakh93/deberta_finetuned_pii')
list(tokenizer.vocab.items())[:10]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

[('Decre', 46136),
 ('Ġcompound', 10624),
 ('ĠTech', 4569),
 ('Ġethnic', 7289),
 ('oter', 31718),
 ('maps', 44754),
 ('ason', 12231),
 ('ĠOber', 22828),
 ('Ġsupervisors', 19971),
 ('Ġstriving', 25514)]

In [11]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
import evaluate
import torch

# Define and load the tokenizer and dataset
tokenizer = AutoTokenizer.from_pretrained('lakshyakh93/deberta_finetuned_pii')
# train_set1_selected = train_set1.select(range(5))

# Define label mapping
# label_map = {'O': 0, 'TIME': 1, 'DATE': 2, 'LASTNAME1': 3, 'LASTNAME2': 4, 'EMAIL': 5, 'SOCIALNUMBER': 6}
label_map = {'O': 0, 'FIRSTNAME_1': 1, 'LASTNAME_1': 2, 'EMAIL_1': 3, 'PHONENUMBER_1': 4, 'SSN_1': 5, 'JOBTITLE_1': 6,
             'SEX_1': 7, 'BUILDINGNUMBER_1': 8, 'STREET_1': 9, 'DOB_1': 10, 'USERNAME_1': 11, 'AGE_1': 12,
             'PREFIX_1': 13, 'ACCOUNTNUMBER_1': 14}


# Function to tokenize and align labels
def tokenize_and_align_labels_train(example):
    # tokenized_inputs = tokenizer(example['source_text'], truncation=True, padding='max_length', max_length=512)
    tokenized_inputs = tokenizer(example['unmasked_text'], truncation=True, padding='max_length', max_length=512)
    labels = ['O'] * len(tokenized_inputs['input_ids'])

    # Adjust indexing to fit the tokenization
    # example['source_text'] = example['source_text'][:512]  # Ensure source_text matches max_length
    example['unmasked_text'] = example['unmasked_text'][:512]  # Ensure source_text matches max_length

    # Extract spans from masked_text
    masked_text = example['masked_text']
    # Find all labels in masked_text using regex
    matches = re.finditer(r'\[([A-Z_]+)\]', masked_text)

    # Create spans and assign labels
    for match in matches:
        label_name = match.group(1)  # Get the label from the match
        start = match.start()
        end = match.end()
        label_id = label_map.get(label_name, 0)

    # for span in example['privacy_mask']:
    #     start, end, label_name = span['start'], span['end'], span['label']
    #     label_id = label_map.get(label_name, 0)

        # span_tokens = tokenizer(example['source_text'][start:end], add_special_tokens=False).tokens()
        span_tokens = tokenizer(example['unmasked_text'][start:end], add_special_tokens=False).tokens()

        # Properly create and align labels (assuming BERT tokenizer)
        for i, token in enumerate(tokenized_inputs.tokens()):
            if token in span_tokens:
                labels[i] = label_id

    numeric_labels = [label_map.get(label, -100) for label in labels]
    numeric_labels += [-100] * (len(tokenized_inputs['input_ids']) - len(numeric_labels))

    tokenized_inputs['labels'] = numeric_labels
    return tokenized_inputs


In [12]:
# Function to tokenize and align labels
def tokenize_and_align_labels_val(example):
    tokenized_inputs = tokenizer(example['source_text'], truncation=True, padding='max_length', max_length=512)
    labels = ['O'] * len(tokenized_inputs['input_ids'])

    # Adjust indexing to fit the tokenization
    example['source_text'] = example['source_text'][:512]  # Ensure source_text matches max_length

    # Create spans and assign labels
    for span in example['privacy_mask']:
        start, end, label_name = span['start'], span['end'], span['label']
        label_id = label_map.get(label_name, 0)

        span_tokens = tokenizer(example['source_text'][start:end], add_special_tokens=False).tokens()

        # Properly create and align labels (assuming BERT tokenizer)
        for i, token in enumerate(tokenized_inputs.tokens()):
            if token in span_tokens:
                labels[i] = label_id

    numeric_labels = [label_map.get(label, -100) for label in labels]
    numeric_labels += [-100] * (len(tokenized_inputs['input_ids']) - len(numeric_labels))

    tokenized_inputs['labels'] = numeric_labels
    return tokenized_inputs

In [14]:
# Apply function to dataset
# encoded_dataset = train_set1_selected.map(tokenize_and_align_labels, batched=False)
encoded_dataset = train_set2.map(tokenize_and_align_labels_train, batched=False)

# Convert to PyTorch tensors
input_ids = torch.tensor(encoded_dataset['input_ids'])
attention_mask = torch.tensor(encoded_dataset['attention_mask'])
labels = torch.tensor(encoded_dataset['labels'])

# Load model
model = AutoModelForTokenClassification.from_pretrained('lakshyakh93/deberta_finetuned_pii')

# Define metric computation
metric = evaluate.load('accuracy')  # You may want to load relevant metrics for token classification

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=2)  # Note: axis=2 for token classification
    true_labels = labels != -100  # Masking out unnecessary labels

    # Flatten predictions and true_labels to compute accuracy
    flattened_predictions = predictions[true_labels]
    flattened_labels = labels[true_labels]

    results = metric.compute(references=flattened_labels, predictions=flattened_predictions)
    return results


# Tokenize validation set
small_val = val_set.shuffle(seed=123).select(range(40))
encoded_small_val = small_val.map(tokenize_and_align_labels_val, batched=False)

# Training arguments
train_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-2,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=10,
    eval_steps=10,
    gradient_accumulation_steps=2,  # Adjust if needed
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_small_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
# Run training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
0,1.941200,0.000000,1.000000
2,0.000000,0.000000,1.000000


TrainOutput(global_step=36, training_loss=0.5392323070102267, metrics={'train_runtime': 894.8162, 'train_samples_per_second': 0.671, 'train_steps_per_second': 0.04, 'total_flos': 175710303682560.0, 'train_loss': 0.5392323070102267, 'epoch': 2.88})

In [16]:
trainer.evaluate()

{'eval_loss': 0.0,
 'eval_accuracy': 1.0,
 'eval_runtime': 2.22,
 'eval_samples_per_second': 18.018,
 'eval_steps_per_second': 2.252,
 'epoch': 2.88}

In [ ]:
# Define label mapping (example mapping)
id2label = model.config.id2label  # Adjust if needed based on your model

# Placeholder mapping for entities
entity_to_placeholder = {
    "PREFIX": "[PREFIX_1]",
    "FIRSTNAME": "[FIRSTNAME_1]",
    "LASTNAME": "[LASTNAME_1]",
    "MEDICAL": "[MEDICAL_1]",
    "BUILDINGNUMBER": "[BUILDINGNUMBER_1]",
    "STREET": "[STREET_1]",
    "CITY": "[CITY_1]",
    "DATE": "[DATE_1]",
    "EMAIL": "[EMAIL_1]",
    "PHONENUMBER": "[PHONENUMBER_1]",
    "AMOUNT": "[AMOUNT_1]",
}

def predict_entities(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits.argmax(dim=-1)

    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    predicted_labels = [id2label[label_id.item()] for label_id in predictions[0]]

    # Post-process to replace recognized entities with placeholders
    redacted_text = ""
    for token, label in zip(tokens, predicted_labels):
        if token in ["[CLS]", "[SEP]", "[PAD]"]:
            continue  # Skip special tokens
        if label != "O":  # Non-neutral labels
            redacted_text += f" {entity_to_placeholder.get(label, token)}"
        else:
            redacted_text += f" {token}"

    return redacted_text.strip()

# Test with auto-generated entries
test_texts = [
    "Hello Mr. John Doe, your meeting is scheduled at 123 Main Street, New York on 01/01/2024.",
    "Please contact Jane Doe at janedoe@example.com for more details."
]

for text in test_texts:
    print(f"\nInput text: {text}")
    redacted_text = predict_entities(text)
    print(f"Redacted output: {redacted_text}")